# Setup

lsgraph was created with the aim of supporting reskilling and upskilling efforts for both individuals and organisations. lsgraph provides services to: 

* collect course details and associate with skills, 
* recommend courses for a learner, 
* recommend job profiles, 
* suggest reskilling options across a workforce.

This notebook walks through the process of setting up skills graphs, organisations, profiles and users.

In [26]:
import json
import os
import pandas as pd
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1"

# Create graph

A skills graph is a linked collection of skills that aids navigation and disambiguates skill names with multiple meanings. 

Within the sample data directory is a graph in nested dictionary format. 

In [13]:
with open('sample_data/graph.json') as fp:
    skill_graph = json.load(fp)
    
# View top level skills
print("The top level skills are:", skill_graph.keys())

The top level skills are: dict_keys(['personal skills', 'art & design', 'certifications', 'data science', 'software engineering', 'information technology', 'engineering', 'natural sciences', 'languages', 'business & management', 'social sciences', 'mathematics', 'computer science'])


In [14]:
# Create a new skills graph in lsgraph

graph_tag = "test"

new_graph = {"name":"Test graph",
             "graph_tag":graph_tag,
             "description":"",
             "graph":skill_graph,
            }

r = requests.post(base_url + "/graphs", json=new_graph)

print("Status code", r.status_code)

print("Response", r.json())
      
graph_id = r.json()["id"]

Status code 200
Response {'id': 1}


In [15]:
# View all graphs

r = requests.get(base_url + "/graphs")
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'graph_tag': 'test', 'id': 1, 'name': 'Test graph'}]


Detailed information on a skills graph can be downloaded.

This is important for accessing the ids of skills. The same nested dictionary format used to load the skills graph is also returned with the addition of a \_meta key that contains the id in a dictionary.

The skills graph can be duplicated by calling /graphs where the \_meta key will be ignored.

In [16]:
r = requests.get(base_url + "/graphs/{0}".format(graph_id))
print("Status code", r.status_code)

print("Response", r.json())

skill_graph = r.json()["graph"]

Status code 200
Response {'created_at': 'Tue, 10 Nov 2020 18:57:05 GMT', 'description': '', 'graph': {'art & design': {'_meta': {'id': 29}, 'craft': {'_meta': {'id': 30}}, 'graphic design': {'_meta': {'id': 31}}, 'spatial design': {'_meta': {'id': 32}, 'exhibition design': {'_meta': {'id': 33}}, 'set design': {'_meta': {'id': 34}}}, 'typography': {'_meta': {'id': 35}}, 'web design': {'_meta': {'id': 36}, 'responsive design': {'_meta': {'id': 37}}, 'wireframing': {'_meta': {'id': 38}}}}, 'business & management': {'_meta': {'id': 374}, 'finance': {'_meta': {'id': 375}}, 'innovation': {'_meta': {'id': 376}}, 'marketing': {'_meta': {'id': 378}}, 'project management': {'_meta': {'id': 377}}, 'sales': {'_meta': {'id': 379}}}, 'certifications': {'BSc': {'_meta': {'id': 70}, 'computer science': {'_meta': {'id': 71}}}, 'IT': {'CompTIA A+': {'_meta': {'id': 41}}, 'MCSA Solutions Associate': {'_meta': {'id': 42}}, 'MCSA: Windows Server 2012': {'_meta': {'id': 43}}, 'MCSE Solutions Expert': {'_met

# Create organisation

An organisation is a collection of users that share a skills graph and profiles.

In [17]:
# Create a new organisation

new_org = {"name":"Test organisation",
             "graph":graph_tag,
            }

r = requests.post(base_url + "/organisations", json=new_org)
print("Status code", r.status_code)
print("Response", r.json())

organisation_id = r.json()["id"]

Status code 200
Response {'id': 1}


In [18]:
# View all organisations

r = requests.get(base_url + "/organisations")
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 1, 'name': 'Test organisation'}]


In [19]:
# View more detail on one organisation

r = requests.get(base_url + "/organisations/{0}".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'created_at': 'Tue, 10 Nov 2020 18:57:13 GMT', 'graph': 'test', 'id': 1, 'name': 'Test organisation', 'updated_at': 'Tue, 10 Nov 2020 18:57:13 GMT'}


# Create profiles

Profiles are used to collect skills and associate levels.

A variety of sample profiles for specific jobs are available

In [43]:
# Load sample data

profile_dir = 'sample_data/profiles/'

level_map = {'B':'beginner','I':'intermediate','A':'advanced','E':'expert'}

def skill_id_map(graph, path):
    if len(path) == 1:
        return graph[path[0]]['_meta']['id']
    return skill_id_map(graph[path[0]], path[1:])

profiles = {}

for fn in os.listdir(profile_dir):
    profile_name = fn[:-len('.skills.csv')]
    data = pd.read_csv(os.path.join(profile_dir, fn), header=None)
    profile_competences = {}
    for _,competence in data.iterrows():
        level = competence[0]
        path=[i for i in competence[1:].tolist() if isinstance(i, str)]
        skill_id = skill_id_map(skill_graph, path)
        profile_competences[skill_id] = level_map[level]
    profiles[profile_name] = profile_competences

In [44]:
profiles

{'Applications Engineer': {16: 'intermediate',
  181: 'advanced',
  203: 'advanced',
  221: 'expert',
  319: 'advanced',
  327: 'advanced',
  377: 'intermediate'},
 'Business Intelligence Analyst': {6: 'advanced',
  19: 'advanced',
  75: 'expert',
  226: 'advanced',
  291: 'advanced'},
 'Chief Information Security Officer': {20: 'advanced',
  301: 'advanced',
  303: 'expert',
  304: 'advanced',
  307: 'expert',
  308: 'expert',
  309: 'expert',
  310: 'expert',
  311: 'expert',
  327: 'expert',
  331: 'advanced',
  332: 'advanced'},
 'Cloud architect': {181: 'advanced',
  184: 'advanced',
  185: 'advanced',
  314: 'advanced',
  319: 'expert',
  327: 'advanced',
  335: 'expert',
  336: 'intermediate',
  337: 'intermediate',
  346: 'advanced'},
 'Computer Hardware Engineers': {20: 'advanced',
  175: 'advanced',
  176: 'intermediate',
  190: 'expert',
  191: 'advanced',
  192: 'expert',
  193: 'advanced',
  194: 'advanced',
  198: 'advanced',
  199: 'advanced',
  200: 'advanced',
  231: '

In [51]:
# Load all profiles

for profile_name, competences in profiles.items():
    print("Creating profile", profile_name)
    new_profile = {"name":profile_name,
                   "competences":competences
                  }
    r = requests.post(base_url + "/organisations/{0}/profiles".format(organisation_id), json=new_profile)
    print("Status code", r.status_code)
    print("Response", r.json())

Creating profile Applications Engineer
Status code 200
Response {'id': 3}
Creating profile Data Scientist
Status code 200
Response {'id': 4}
Creating profile Computer and Information Systems Managers
Status code 200
Response {'id': 5}
Creating profile Web Developers
Status code 200
Response {'id': 6}
Creating profile Software Engineer
Status code 200
Response {'id': 7}
Creating profile Software Engineer ML
Status code 200
Response {'id': 8}
Creating profile Systems Engineer
Status code 200
Response {'id': 9}
Creating profile DevOps Engineer
Status code 200
Response {'id': 10}
Creating profile Machine Learning Researcher
Status code 200
Response {'id': 11}
Creating profile Business Intelligence Analyst
Status code 200
Response {'id': 12}
Creating profile Software Developers, Applications
Status code 200
Response {'id': 13}
Creating profile Cloud architect
Status code 200
Response {'id': 14}
Creating profile Sales Engineer
Status code 200
Response {'id': 15}
Creating profile Database Adm

In [60]:
# List all profiles for an organisation

r = requests.get(base_url + "/organisations/{0}/profiles".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 1, 'name': 'Test organisation'}, {'id': 2, 'name': 'Applications Engineer'}, {'id': 3, 'name': 'Applications Engineer'}, {'id': 4, 'name': 'Data Scientist'}, {'id': 5, 'name': 'Computer and Information Systems Managers'}, {'id': 6, 'name': 'Web Developers'}, {'id': 7, 'name': 'Software Engineer'}, {'id': 8, 'name': 'Software Engineer ML'}, {'id': 9, 'name': 'Systems Engineer'}, {'id': 10, 'name': 'DevOps Engineer'}, {'id': 11, 'name': 'Machine Learning Researcher'}, {'id': 12, 'name': 'Business Intelligence Analyst'}, {'id': 13, 'name': 'Software Developers, Applications'}, {'id': 14, 'name': 'Cloud architect'}, {'id': 15, 'name': 'Sales Engineer'}, {'id': 16, 'name': 'Database Administrators'}, {'id': 17, 'name': 'Security Engineer'}, {'id': 18, 'name': 'Computer Hardware Engineers'}, {'id': 19, 'name': 'Data Engineer'}, {'id': 20, 'name': 'Product Manager'}, {'id': 21, 'name': 'UX Designer'}, {'id': 22, 'name': 'Solutions Architect'}, {'id': 23, 'name

# Create users

In [58]:
# Create four users with the competences associated with a job profile

users = [("Adam", profiles["Data Engineer"]), 
         ("Beth", profiles["Electrical Engineer"]), 
         ("Chris", profiles["Mobile Developer"]), 
         ("Dani", profiles["Security Engineer"])]

for user,competences in users:
    print("Creating user", user)
    new_user = {"email":"{0}@test.local".format(user),
                "name":user,
                "graph":graph_tag,
                "competences":competences
               }
    r = requests.post(base_url + "/organisations/{0}/users".format(organisation_id), json=new_user)
    print("Status code", r.status_code)
    print("Response", r.json())

Creating user Adam
Status code 200
Response {'id': 3}
Creating user Beth
Status code 200
Response {'id': 4}
Creating user Chris
Status code 200
Response {'id': 5}
Creating user Dani
Status code 200
Response {'id': 6}


In [61]:
# List all users for an organisation

r = requests.get(base_url + "/organisations/{0}/users".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 3, 'name': 'Adam'}, {'id': 4, 'name': 'Beth'}, {'id': 5, 'name': 'Chris'}, {'id': 6, 'name': 'Dani'}]
